# The aixACCT Notebook

### Table of contents


In [ ]:
%matplotlib widget
# This imports the functions I implemented
try:
    from brlopack import brlopack
    print("Successfully loaded `brlopack` from `brlopack`")
except:
    print("!!!!!!!!!!!!!")
    print("ERROR: Importing the package was NOT successful.")
# import pandas as pd

### 1. Select your files and load data from them
We're using the `browseDirectories` method of the `brlopack` class ~~(`tkinter` GUI - specifically the `askopenfilename()` function)~~

In [ ]:
paket = brlopack()
paket.browseDirectories()

In [ ]:
paket.loadFiles()

### 2. Peek into your data using `tellMeFiles`, `tellMeTablesInFile` or `tellMeColumnsInTable`.

The data has loaded successfully.  
Now you can always check which files have been loaded using the `tellMeFiles()` method.

In [ ]:
paket.tellMeFiles()

Now you can check which tables (`DataFrames`) have been loaded from that specific file using the `tellMeTablesInFile` method.

In [ ]:
paket.tellMeTablesInFile("C:/Users/Uporabnik/Documents/Git projekti/ajzakt/data/BSFO13_RS800_10-150kVcm_RT.dat")

Now - let's divide the Voltage and Current with the area of the sample

### Skip for now: 3. Manipulate the `.data` field


Keep in mind that raw data itself is easily found under `.data` field.  

However that is too much data to see at once. We can narrow down what we are looking for by specifying the *file name* and *table* (`DataFrame`) *name*.  

Copy paste a *file name* and a *table name* from outputs above and put them in square brackets. We can store this data in a new variable.


In [ ]:
some_uniquely_defined_table = paket.data["C:/Users/Uporabnik/Documents/Git projekti/ajzakt/data/BSFO13_RS800_10-150kVcm_RT.dat"]["Contour 1"]
some_uniquely_defined_table

Even that is a bit much. Maybe it is simply enough to get some basic summary `.info()` of this table, like so:

In [ ]:
some_uniquely_defined_table.info()

Now, if we want to see only some columns of the data - we can just put the column name in a square bracket as well.  

Just copy paste some column name from above.

In [ ]:
some_uniquely_defined_table["P1 [uC/cm2]"]

Here's another one:

In [ ]:
time = some_uniquely_defined_table["Time [s]"]
voltage = some_uniquely_defined_table["V+ [V]"]
p1 = some_uniquely_defined_table["P1 [uC/cm2]"]
john = some_uniquely_defined_table["P2 [uC/cm2]"]

# You can name your variables however you want.
# But you need to copy paste the column titles from above
john 

As you can see above, it again lists out all of the values in that column.  

After that it tells you the columns name and length (how many values you have)  

  
Let's do that again for a different column

In [ ]:
# You can type the name of any of the variables from above to see the values at that column
voltage

You can also do this with another table of the same file; or a different file altogether.

Like always, feel free to play around with different values and see what gets outputted.  

Keep in mind - after loading your data in a `brlopack` object, you can see it within the `.data` field.  
After that the syntax is just adding square brackets that narrow down what you care about - firstly the file, then the table and then the column.  
**Example**: `SOME_BRLOPACK_VARIABLE.data[FILE_NAME][TABLE_NAME][COLUMN_NAME]`

If any errors happen, you can Google them to get an explanation (probably on the *Stack Overflow* website) why that happened and how to avoid it.

### 4. Loaded constants (data before the table)

Let's see the values of constants (like area or thickness) that we loaded from the file

In [ ]:
for file in paket.tellMeFiles():
    for table in paket.tellMeTablesInFile(file):
        print(paket.constants[file][table])

Now we can change the unit of measurement  
(feel free to re-run the code cell from above to see the results of `changeUnitOfConstant`)

In [ ]:
from valueConversion import convertPrefix
def changeUnitOfConstant(package, constantName, unitPrefix):
    for file in package.tellMeFiles():
        for table in package.tellMeTablesInFile(file):
            package.constants[file][table][constantName] = convertPrefix(package.constants[file][table][constantName], unitPrefix)
    
changeUnitOfConstant(paket, "thickness", "c")

Now we can divide any column with the constant we want using the `divide()` function.  
In the parentheses you should input the following arguments:  
  1. `package` - place where we loaded our data  
  2. `columnName` - the column we want to divide  
  3. `newColumnName` - where to place newly derived data  
  4. `constName` - the constant from above we want to use for dividing  

In the end you should get something like `divide(package, col1, col2, const)`

In [ ]:
def divide(package, columnName, newColumnName, constName):
    for file in paket.tellMeFiles():
        for table in paket.tellMeTablesInFile(file):
            (constantVal, constantUnit) = paket.constants[file][table][constName]
            paket.data[file][table][newColumnName] = [el/constantVal for el in paket.data[file][table][columnName]]

divide(paket, "V+ [V]", "E+ [V/cm]", "thickness") 
divide(paket, "V- [V]", "E- [V/cm]", "thickness") 

We have successfully calculated the electric field.  

We can repeat this procedure for dividing the D column with thickness to get the relative change.  

In [ ]:
divide(paket, "D3 [nm]", "delta3 [%]", "thickness")
divide(paket, "D2 [nm]", "delta2 [%]", "thickness")

Here is how we can view the contents of this newly made column

In [ ]:
for el in paket.data[file][table]["delta2 [%]"]:
    #if el > 10**-5:
    print(el)

The package `arrayConversion` has a useful function called `adapter_ConvertPrefix` which allows us to change kg to g; GW to kW, km to nm etc, assuming data in a list.

In [ ]:
import arrayConversion
arrayConversion.adapter_ConvertPrefix(paket, "delta2 [%]", "delta2 [c%]")

Now we can get a list of columns in all tables:

In [ ]:
for f in paket.tellMeFiles():
    for t in paket.tellMeTablesInFile(f):
        #print(paket.data[f][t]["delta2 [c%]"])
        paket.data[f][t].info()

## Plotting data

Now that we are roughly aware of how data is stored in a `brlopack` object - we can easily plot data.

### 1. Setting up interesting measurements for plotting
Using Python dictionaries - key:value pairs (x axis: y axes)

In [ ]:
# Independent variable
x_axis = "V+ [V]" 

# Dependent variables
y_axes = ["P1 [uC/cm2]", "P2 [uC/cm2]"]

# Pair them up
plots = {
    x_axis: y_axes
    # you can pair up another x axis here; then a ":"; then y_axes
    }

# Let's see what we have stored in memory
plots

In [ ]:
plots = {
    "E+ [V/cm]": ["P2 [uC/cm2]", "delta2 [%]"],
    "E- [V/cm]": ["P3 [uC/cm2]", "delta3 [%]"]
}

### 2. Printing only some contours

What if we don't want to plot all contours?

Run the cell below to get a list of contours and a checkbox to exclude irrelevant ones.

In [ ]:
#from helperFunctions import newestSplit, splitMessage, checkedBoxes
paket.resetPlotSettings()
def changed(b):
    #i = splitMessage.index(b["owner"].description)
    checkedBoxes[b["owner"].description] = b["owner"].value
    global paket
    fileName = b["owner"].description
    truthValue = b["owner"].value
    paket.shouldIPlotFile(fileName, truthValue)
    print("\n========\n")
    print("{}: switched to {}".format(b["owner"].description, b["owner"].value))
    
    for i, key in enumerate(checkedBoxes):
        if key == b["owner"].description: break
    
    for chkBx2 in checkboxes2[i]:
        chkBx2.set(b["owner"].value)

def sub_changed(b):
    global paket
    table, file = b["owner"].description.split("|")
    print("Tbl: {} fl: {}".format(table, file))
    paket.shouldIPlotTables(file, table, b["owner"].value)
    print("\n========\n")
    print("{}: subswitched to {}".format(b["owner"].description, b["owner"].value))
    
from IPython.display import display
from ipywidgets import Checkbox

#print(contours)
checkboxes = []; checkboxes2 = []
checkedBoxes = {}
print(checkedBoxes)

for fileName in paket.tellMeFiles():
    checkboxes.append(Checkbox(True, description=str(fileName)))
    checkedBoxes[fileName] = True
    checkboxes2.append([])
    for table in paket.tellMeTablesInFile(file):
        checkboxes2[-1].append(Checkbox(True, description=table+"|"+fileName))
        #checkedBoxes[table+fileName] = True
        
for i, chkBx in enumerate(checkboxes):
    display(chkBx)
    chkBx.observe(changed, names=['value'])
    for chkBx2 in checkboxes2[i]:
        display(chkBx2)
        chkBx2.observe(sub_changed, names=['value'])
    
#TODO make a function that import/exports bool values to and from checkboxes

Let's see how our variables store this data (for checked files)

In [ ]:
checkedBoxes

Let's see how our variables store this data (for checked tables of a file)

In [ ]:
paket.plotFiles

Now that we are confident what gets plotted and what doesn't, we can run the cell below

In [ ]:
desirable = [key for key in checkedBoxes if checkedBoxes[key]]
for x_axis in plots:
    for y_axis in plots[x_axis]:
        paket.plotData(x_axis, y_axis, desirable)

## Exporting to excel

It is implemented in the `brlopack` using the `exportToExcel()` method

In [ ]:
paket.exportToExcel()